# QED-to-Lattice Closure Loop — Phase II Deliverable (2025-10-28T03:50:08)

Objective: One deterministic pipeline (one normalization, no knob-turning) that:
1. Takes **Bridge** outputs { ω_c, m_Γ, χ }.
2. Computes stiffnesses **K₁, K₂, K₃** (U(1), SU(2), SU(3)).
3. Runs 1‑loop (and optionally 2‑loop) RGEs down to **M_Z** to get **α(M_Z)**, **sin²θ_W(M_Z)**, **α_s(M_Z)**.
4. Predicts **lattice spacing** `a` and **string tension** `σ` via **MATH‑YM‑003** mapping.
5. Compares to **PDG** (couplings) and **FLAG/MILC** (a, σ), without retuning.

> This notebook is a scaffold with explicit computational steps. You can lock the mapping (one normalization) and press **Run All**.

In [1]:
# Imports
import numpy as np
import pandas as pd
from math import log, pi, sqrt

# Constants (can be adjusted if you want a different input scale or reference values)
MZ_GeV = 91.1876  # Z boson mass (GeV)

# PDG-like targets at MZ (reference values for comparison)
PDG_alpha_em_inv = 127.955  # alpha_em^-1(MZ)
PDG_sin2_thetaW = 0.23122   # sin^2 theta_W(MZ)
PDG_alpha_s = 0.1179        # alpha_s(MZ)

targets = {
    "alpha_em_inv_MZ": PDG_alpha_em_inv,
    "sin2_thetaW_MZ": PDG_sin2_thetaW,
    "alpha_s_MZ": PDG_alpha_s,
}

targets

{'alpha_em_inv_MZ': 127.955, 'sin2_thetaW_MZ': 0.23122, 'alpha_s_MZ': 0.1179}

## Step 0 — Inputs from the Bridge

> Provide **ω_c [Hz]**, **m_Γ [MeV]**, **χ [dimensionless susceptibility]**.
> Choose a **Bridge scale** Λ_B (GeV) at which the stiffness dictionary is defined.

In [2]:
# --- USER INPUTS (Bridge) ---
omega_c = 1.0e23    # Hz
m_Gamma_MeV = 17.0  # MeV
chi = 0.85          # unitless

# Bridge normalization scale (GeV) — where K_i are defined via the dictionary
Lambda_B_GeV = 1000.0  # 1 TeV default; adjust if your Bridge prefers a different Λ_B

omega_c, m_Gamma_MeV, chi, Lambda_B_GeV

(1e+23, 17.0, 0.85, 1000.0)

## Step 1 — Stiffness dictionary (MATH‑QED‑001…005, MATH‑YM‑001…003)

**One normalization, no knob-turning**: define a single scale `Λ_0` and fixed, explicit forms that map `{ω_c, m_Γ, χ}` → `{K₁, K₂, K₃}`.

> **Example (placeholder)**:  
> We treat the stiffnesses as
> \[
> K_1 = \Lambda_0 \left(
rac{\omega_c}{\omega_0}
ight)^{1/3}\chi^{\,p_1},\quad
> K_2 = \Lambda_0 \left(
rac{m_\Gamma}{m_0}
ight)^{1/2}\chi^{\,p_2},\quad
> K_3 = \Lambda_0 \left(
rac{\omega_c m_\Gamma}{\omega_0 m_0}
ight)^{1/4}\chi^{\,p_3}.
> \]
> with a **single** normalization Λ₀ and **fixed exponents** p₁,p₂,p₃ chosen from theory (no fitting here).
> 
> Then define **gauge couplings at Λ_B** by a proportional map \( lpha_i(\Lambda_B) \propto 1/K_i^2 \) up to the **single** normalization.
> 
> Replace the placeholders below with your final forms *once fixed in the paper text*.

In [3]:
# --- SINGLE NORMALIZATION AND FIXED FORMS (PLACEHOLDER THEORY CHOICE) ---
Lambda0 = 1.0  # single normalization [arbitrary units], set to 1.0 for a pure no-knob baseline

# Reference scales to make the ratios dimensionless
omega0 = 1.0e23      # Hz
m0_MeV = 17.0        # MeV

# Fixed exponents from theory choice (no fitting)
p1, p2, p3 = 0.0, 0.0, 0.0

def K1(omega_c, mGamma_MeV, chi):
    return Lambda0 * (omega_c/omega0)**(1/3) * (chi**p1)

def K2(omega_c, mGamma_MeV, chi):
    return Lambda0 * (mGamma_MeV/m0_MeV)**(1/2) * (chi**p2)

def K3(omega_c, mGamma_MeV, chi):
    return Lambda0 * ((omega_c/omega0)*(mGamma_MeV/m0_MeV))**(1/4) * (chi**p3)

K1_val, K2_val, K3_val = K1(omega_c, m_Gamma_MeV, chi), K2(omega_c, m_Gamma_MeV, chi), K3(omega_c, m_Gamma_MeV, chi)
K1_val, K2_val, K3_val

(1.0, 1.0, 1.0)

### Map stiffness → gauge couplings at Λ_B

We define (placeholder mapping; replace with MATH‑YM‑002 canonical form once fixed):
\[
\alpha_1(\Lambda_B) = \frac{c}{K_1^2},\quad
\alpha_2(\Lambda_B) = \frac{c}{K_2^2},\quad
\alpha_3(\Lambda_B) = \frac{c}{K_3^2},
\]
with a **single** constant `c` determined by the **single normalization choice** (e.g., fix one observable once, then never touch `c` again).

In [4]:
# Fix c by one reference normalization: we choose to match α_em(MZ) after running (deterministic closure).
# This block determines c ONCE by requiring the pipeline to reproduce alpha_em(MZ) target (no tuning elsewhere).

# 1-loop beta coefficients for SM (dg/dlnμ = (b/16π^2) g^3)
# For α^{-1}: d(α^{-1})/dlnμ = - b/(2π)
b1 = 41/6    # U(1)_Y (GUT-normalized)
b2 = -19/6   # SU(2)_L
b3 = -7      # SU(3)_c

def alpha_inv_run_1loop(alpha_inv_mu0, b, mu, mu0):
    return alpha_inv_mu0 - (b/(2*pi)) * np.log(mu/mu0)

def run_to_MZ(alpha1_LB, alpha2_LB, alpha3_LB, Lambda_B_GeV, MZ_GeV):
    a1_inv = 1.0/alpha1_LB
    a2_inv = 1.0/alpha2_LB
    a3_inv = 1.0/alpha3_LB
    a1_inv_MZ = alpha_inv_run_1loop(a1_inv, b1, MZ_GeV, Lambda_B_GeV)
    a2_inv_MZ = alpha_inv_run_1loop(a2_inv, b2, MZ_GeV, Lambda_B_GeV)
    a3_inv_MZ = alpha_inv_run_1loop(a3_inv, b3, MZ_GeV, Lambda_B_GeV)
    return 1.0/a1_inv_MZ, 1.0/a2_inv_MZ, 1.0/a3_inv_MZ

def electroweak_from_hyper_isospin(alpha1, alpha2):
    # GUT normalization: α1 = (5/3) α_Y
    alpha_Y = (3/5) * alpha1
    # α_em = α2 * α_Y / (α2 + α_Y)
    alpha_em = (alpha2*alpha_Y) / (alpha2 + alpha_Y)
    sin2_thetaW = alpha_Y / (alpha2 + alpha_Y)
    return alpha_em, sin2_thetaW

# Determine c by matching α_em(MZ) to PDG target (single normalization)
def determine_c(K1_val, K2_val, K3_val):
    # α_i(Λ_B) = c / K_i^2  => α1, α2, α3 are proportional to c
    # After running to MZ, α_em(MZ) must equal PDG target.
    # Solve for c using a simple monotonic 1D search.
    target_alpha_em = 1.0 / targets["alpha_em_inv_MZ"]
    c_lo, c_hi = 1e-8, 1e+2
    for _ in range(80):
        c_mid = (c_lo + c_hi)/2.0
        a1_LB = c_mid / (K1_val**2)
        a2_LB = c_mid / (K2_val**2)
        a3_LB = c_mid / (K3_val**2)
        a1_MZ, a2_MZ, a3_MZ = run_to_MZ(a1_LB, a2_LB, a3_LB, Lambda_B_GeV, MZ_GeV)
        aem, s2w = electroweak_from_hyper_isospin(a1_MZ, a2_MZ)
        if aem > target_alpha_em:
            c_hi = c_mid
        else:
            c_lo = c_mid
    return (c_lo + c_hi)/2.0

c_norm = determine_c(K1_val, K2_val, K3_val)
c_norm

0.021363909422278192

## Step 2 — Run RGEs down to M_Z and compare to PDG

In [5]:
# Compute gauge couplings at Λ_B
alpha1_LB = c_norm / (K1_val**2)
alpha2_LB = c_norm / (K2_val**2)
alpha3_LB = c_norm / (K3_val**2)

# Run to MZ
alpha1_MZ, alpha2_MZ, alpha3_MZ = run_to_MZ(alpha1_LB, alpha2_LB, alpha3_LB, Lambda_B_GeV, MZ_GeV)

# Derived electroweak
alpha_em_MZ, sin2_thetaW_MZ = electroweak_from_hyper_isospin(alpha1_MZ, alpha2_MZ)

res_RGE = {
    "alpha_em_inv_MZ (pred)": 1.0/alpha_em_MZ,
    "sin2_thetaW_MZ (pred)": sin2_thetaW_MZ,
    "alpha_s_MZ (pred)": alpha3_MZ,
}
res_RGE

{'alpha_em_inv_MZ (pred)': np.float64(127.95499999999996),
 'sin2_thetaW_MZ (pred)': np.float64(0.35638261283445133),
 'alpha_s_MZ (pred)': np.float64(0.0226552588387174)}

In [6]:
# Compare to targets
compare = pd.DataFrame({
    "quantity": ["alpha_em_inv_MZ", "sin2_thetaW_MZ", "alpha_s_MZ"],
    "pred": [1.0/alpha_em_MZ, sin2_thetaW_MZ, alpha3_MZ],
    "target": [targets["alpha_em_inv_MZ"], targets["sin2_thetaW_MZ"], targets["alpha_s_MZ"]]
})
compare["delta"] = compare["pred"] - compare["target"]
compare

,quantity,pred,target,delta
0,alpha_em_inv_MZ,127.955000,127.95500,-4.263256e-14
1,sin2_thetaW_MZ,0.356383,0.23122,1.251626e-01
2,alpha_s_MZ,0.022655,0.11790,-9.524474e-02


## Step 3 — Predict lattice observables (MATH‑YM‑003)

We adopt the working relation (user-specified in your module text):
\[
\sigma \sim \frac{\kappa_3}{\xi_\Gamma^2},\qquad a \sim f(\alpha_3(M_Z), \Lambda_{QCD}, ...)
\]
Provide explicit definitions for \(\kappa_3\) and \(\xi_\Gamma\) in terms of \{ω_c, m_Γ, χ\} or the derived K_i, then compute:
- **string tension** \(\sigma\) (compare \(\sqrt{\sigma}\approx 440\,\mathrm{MeV}\)),
- **lattice spacing** `a` (compare to FLAG/MILC tables for chosen action).

In [7]:
# === Step 3.1: Inputs from the Solver Run (from qed_option_c_report.json) ===

# These are the direct outputs from your plateau binding simulation
phi_star = 0.15
Delta_phi_eff = 1.44
chi = 1.0

# These are the resulting lattice-unit observables from that same run
sqrt_sigma_lattice = 207.57157524029304
r0_lattice = 0.0061883389205843724
a_lattice = 0.001167611117091391

# This is the crucial curvature of H(φ) at φ*, needed for an accurate κ₃
# As per the report, this value reproduces the observed sqrt_sigma_lattice
curvature_H = 0.2366782407407408

print("Option C inputs loaded successfully.")

Option C inputs loaded successfully.


In [8]:
# --- PLACEHOLDER MAPPINGS (replace with your canonical MATH-YM-003 definitions) ---
# Example proxies:
# kappa_3 from stiffness K3; xi_Gamma from (omega_c, m_Gamma, chi) as a coherence length scale.
# Units are illustrative; rescale once canonical mapping is finalized.

def kappa3_from_K3(K3):  # dimensionless proxy
    return K3

def xi_Gamma_from_bridge(omega_c, mGamma_MeV, chi):  # in fm (just a stand-in)
    # Coherence length ~ inverse of an energy scale; here a toy proxy
    # 1 MeV^-1 ≈ 197.3 fm; use m_Gamma to set a scale, modulated by susceptibility.
    return (197.3269804 / max(mGamma_MeV, 1e-6)) * (1.0/ max(chi, 1e-6))

kappa3 = kappa3_from_K3(K3_val)
xi_Gamma_fm = xi_Gamma_from_bridge(omega_c, m_Gamma_MeV, chi)

# String tension sigma ~ kappa3 / xi_Gamma^2
sigma_MeV2 = kappa3 / ( (197.3269804/xi_Gamma_fm)**2 )  # very rough, illustrative
sqrt_sigma_MeV = sqrt(abs(sigma_MeV2)) if sigma_MeV2>0 else float("nan")

# Lattice spacing: exemplar relation ~ xi_Gamma scaled by a factor of α_s
a_fm = xi_Gamma_fm * (alpha3_MZ / 0.12)  # scale so that α_s≈0.12 sets a≈xi_Gamma (toy)

{
    "kappa3": kappa3,
    "xi_Gamma_fm": xi_Gamma_fm,
    "sqrt_sigma_MeV (pred)": sqrt_sigma_MeV,
    "a_fm (pred)": a_fm
}

{'kappa3': 1.0,
 'xi_Gamma_fm': 11.607469435294117,
 'sqrt_sigma_MeV (pred)': 0.058823529411764705,
 'a_fm (pred)': np.float64(2.191418537659076)}

In [9]:
# === Step 4.1: Drop-in cell for the Option C end-to-end closure pipeline ===
from dataclasses import dataclass
from typing import Callable, Optional, Tuple, Dict
import math

HBARC_MEV_FM = 197.3269804

@dataclass
class ClosureInputs:
    # Solver-side
    phi_star: float
    Delta_phi_eff: float
    chi: float = 1.0
    # Optional existing lattice outputs for calibration
    sqrt_sigma_lattice: Optional[float] = None
    r0_lattice: Optional[float] = None
    a_lattice: Optional[float] = None
    # Physical calibration target
    target_sqrt_sigma_MeV: float = 440.0

def omega_c_from_phi_star(phi_star: float) -> float:
    return 1.0 / phi_star

def xi_gamma(omega_c: float, Delta_phi_eff: float, chi: float=1.0) -> float:
    return (chi**-0.5) / (omega_c * Delta_phi_eff)

def kappa3_from_curvature(omega_c: float, phi_star: float, curvature: float) -> float:
    return (omega_c / phi_star) ** 2 * curvature

def sigma_from_components(kappa3_val: float, xi_gamma_val: float) -> float:
    return kappa3_val / (xi_gamma_val ** 2)

def option_c_closure_end2end(inputs: ClosureInputs, curvature: float) -> Dict:
    # 1) ω_c and ξ_Γ
    omega_c = omega_c_from_phi_star(inputs.phi_star)
    xiG = xi_gamma(omega_c, inputs.Delta_phi_eff, inputs.chi)

    # 2) κ₃ and σ
    k3 = kappa3_from_curvature(omega_c, inputs.phi_star, curvature)
    sigma_val = sigma_from_components(k3, xiG)
    sqrt_sigma_lat = math.sqrt(sigma_val)

    out = {
        "inputs": vars(inputs),
        "derived_lattice": {
            "omega_c": omega_c, "xi_Gamma": xiG, "curvature": curvature,
            "kappa3": k3, "sigma": sigma_val, "sqrt_sigma": sqrt_sigma_lat,
        }, "calibration": None, "converted": None,
    }

    # 3) Physical calibration (optional)
    if inputs.sqrt_sigma_lattice is not None:
        mass_scale = inputs.target_sqrt_sigma_MeV / inputs.sqrt_sigma_lattice
        length_unit = HBARC_MEV_FM / mass_scale
        converted = {}
        if inputs.r0_lattice is not None:
            converted["r0_fm"] = inputs.r0_lattice * length_unit
        if inputs.a_lattice is not None:
            converted["a_fm"] = inputs.a_lattice * length_unit
        out["calibration"] = {
            "mass_scale_MeV_per_unit": mass_scale,
            "length_unit_fm": length_unit
        }
        out["converted"] = converted
    return out

print("Closure pipeline functions defined.")

Closure pipeline functions defined.


In [10]:
# === Step 5.1: Run the pipeline and get final predictions ===

# Create the input object from our solver data
inputs = ClosureInputs(
    phi_star=phi_star,
    Delta_phi_eff=Delta_phi_eff,
    chi=chi,
    sqrt_sigma_lattice=sqrt_sigma_lattice,
    r0_lattice=r0_lattice,
    a_lattice=a_lattice
)

# Run the end-to-end calculation using the known curvature
results = option_c_closure_end2end(inputs, curvature=curvature_H)

# Display the final, physically meaningful predictions
final_predictions = {
    "Predicted r0 (fm)": results["converted"]["r0_fm"],
    "Predicted lattice spacing a (fm)": results["converted"]["a_fm"],
    "Sanity Check: sqrt(sigma) [lattice units]": results["derived_lattice"]["sqrt_sigma"],
}

import json
print(json.dumps(final_predictions, indent=2))

{
  "Predicted r0 (fm)": 0.5760706721099308,
  "Predicted lattice spacing a (fm)": 0.10869257964338316,
  "Sanity Check: sqrt(sigma) [lattice units]": 207.57157524029304
}


In [11]:
# === Step 6.1: Define Stiffnesses using Casimir Scaling ===

# From our Option C run, the stiffness of the SU(3) channel is just kappa3.
# We take this value directly from the previous results.
K3 = results["derived_lattice"]["kappa3"]

# Casimir scaling provides a physics-based first guess for the other stiffnesses.
# C₂(G) is the quadratic Casimir invariant for the fundamental representation of a group G.
C3 = 4/3  # For SU(3)
C2 = 3/4  # For SU(2)
C1 = 0.0   # For U(1)

# To avoid a division by zero for the Abelian U(1) group, we add a small "floor".
epsilon_floor = 0.1

K2 = K3 * (C2 / C3)
K1 = K3 * (C1 + epsilon_floor) / C3

print(f"Stiffnesses (K_i) from Casimir Scaling:")
print(f"  K3 (SU(3)): {K3:.4f} (from Option C)")
print(f"  K2 (SU(2)): {K2:.4f}")
print(f"  K1 (U(1)):  {K1:.4f}")

Stiffnesses (K_i) from Casimir Scaling:
  K3 (SU(3)): 467.5126 (from Option C)
  K2 (SU(2)): 262.9758
  K1 (U(1)):  35.0634


In [12]:
# === Step 7.1: Determine the Single Normalization Constant ===

# We re-use the *exact same* function from earlier in the notebook.
# It finds the one `c_norm` that correctly produces α_em(MZ) after running
# the couplings (which are derived from K₁, K₂, K₃) down from the bridge scale.
c_norm = determine_c(K1, K2, K3)

print(f"Determined single normalization constant: c_norm = {c_norm:.6f}")

# === Step 7.2: Predict sin²θ_W and α_s with Zero Free Parameters ===

# Calculate the high-scale couplings at Λ_B using our stiffnesses and the c_norm
alpha1_LB = c_norm / (K1**2)
alpha2_LB = c_norm / (K2**2)
alpha3_LB = c_norm / (K3**2)

# Run the couplings down to the Z boson mass
alpha1_MZ, alpha2_MZ, alpha3_MZ = run_to_MZ(alpha1_LB, alpha2_LB, alpha3_LB, Lambda_B_GeV, MZ_GeV)

# Calculate the final physical observables
alpha_em_MZ_pred, sin2_thetaW_MZ_pred = electroweak_from_hyper_isospin(alpha1_MZ, alpha2_MZ)
alpha_s_MZ_pred = alpha3_MZ

# --- Final Comparison ---
final_df = pd.DataFrame({
    "Observable": ["alpha_em_inv(MZ)", "sin²θ_W(MZ)", "alpha_s(MZ)"],
    "Prediction": [1.0/alpha_em_MZ_pred, sin2_thetaW_MZ_pred, alpha_s_MZ_pred],
    "Target (PDG)": [targets["alpha_em_inv_MZ"], targets["sin2_thetaW_MZ"], targets["alpha_s_MZ"]]
})
final_df["Error (%)"] = 100 * (final_df["Prediction"] - final_df["Target (PDG)"]) / final_df["Target (PDG)"]

print("\n--- Final Predictions (from Casimir-Scaled Stiffnesses) ---")
print(final_df.to_string(index=False))

Determined single normalization constant: c_norm = 100.000000

--- Final Predictions (from Casimir-Scaled Stiffnesses) ---
      Observable  Prediction  Target (PDG)  Error (%)
alpha_em_inv(MZ)  715.187486     127.95500 458.936725
     sin²θ_W(MZ)    0.965280       0.23122 317.472345
     alpha_s(MZ)    0.000458       0.11790 -99.611465


In [13]:
# === Cell 13 (post 7.1): Gauge-channel stiffnesses and QED/QCD predictions ===
# Date: 2025-10-28
# This cell closes the perturbative half of the loop by mapping Option-C/stiffness geometry to
# gauge couplings with a single normalization, then (optionally) running RGEs to M_Z.

import math, numpy as np
from dataclasses import dataclass
from typing import Optional, Callable, Dict

# ---------- Config ----------
MODE = "casimir"      # "casimir" (fast sanity) or "channels" (preferred, first-principles)
MAPPING = "invK"      # "invK" -> alpha_i = c * chi_i / K_i   ;  "invK2" -> c * chi_i / K_i^2
ALPHA_EM_INV_TARGET = 127.955   # PDG at M_Z
USE_RGE_IF_AVAILABLE = True     # if a function run_to_MZ(...) exists in globals(), use it

# Optional susceptibilities (can be measured via linear response; here default to 1.0)
chi1 = globals().get("chi1", 1.0)
chi2 = globals().get("chi2", 1.0)
chi3 = globals().get("chi3", 1.0)

# ---------- Utilities ----------
def omega_c_from_phi_star(phi_star: float) -> float:
    return 1.0 / phi_star

def local_curvature(energy_fn: Callable[[float], float], phi_star: float, h: float=1e-3) -> float:
    fph = energy_fn(phi_star + h)
    fmh = energy_fn(phi_star - h)
    f0  = energy_fn(phi_star)
    return (fph - 2.0*f0 + fmh)/(h*h)

def stiffness_from(phi_star: float, curvature: float) -> float:
    """K = (omega_c/phi_star)^2 * curvature"""
    oc = omega_c_from_phi_star(phi_star)
    return (oc / phi_star)**2 * curvature

def solve_cnorm_for_alpha_em(K1, K2, chi1, chi2, mapping: str, alpha_em_inv_target: float) -> float:
    # alpha1 = c*chi1/K1^p ; alpha2 = c*chi2/K2^p ; alphaY = (3/5) * alpha1
    p = 1 if mapping == "invK" else 2
    a_em = 1.0 / alpha_em_inv_target
    denom = ( (K2**p)/chi2 + (5.0/3.0)*(K1**p)/chi1 )
    # 1/alpha_em = (K2^p)/(c*chi2) + (5/3)*(K1^p)/(c*chi1)  =>  c = a_em * denom
    return a_em * denom

def couplings_from_K(K1,K2,K3, chi1,chi2,chi3, mapping: str, c_norm: float):
    p = 1 if mapping == "invK" else 2
    a1 = c_norm * chi1 / (K1**p)
    a2 = c_norm * chi2 / (K2**p)
    a3 = c_norm * chi3 / (K3**p)
    return a1,a2,a3

def summary_row(name, pred, target=None):
    if target is None:
        return f"{name:<16} {pred:>12.6f}"
    err = (pred - target)/target * 100.0
    return f"{name:<16} {pred:>12.6f}  vs  {target:<10.6f}  Δ%={err:+8.3f}"

# ---------- Channel stiffness acquisition ----------
@dataclass
class ChannelResult:
    name: str
    phi_star: float
    Delta_eff: float
    curvature: float
    K: float

def get_K_via_optionC_SU3_from_globals() -> Optional[float]:
    """Try to reconstruct K3 from variables defined earlier in the notebook."""
    phi_star = globals().get("phi_star", None)
    curvature = globals().get("curvature", None) or globals().get("curvature_SU3", None)
    if phi_star is not None and curvature is not None:
        return stiffness_from(phi_star, curvature)
    # If the notebook saved κ3 directly under a name:
    if "kappa3" in globals():
        return float(globals()["kappa3"])
    return None

def get_Ki_by_casimir():
    """Fast sanity pass: scale K2,K1 off K3 using fundamental Casimirs and an Abelian floor ε."""
    K3 = get_K_via_optionC_SU3_from_globals()
    if K3 is None:
        # fallback: allow user to have stored a number as K3_optionC
        K3 = float(globals().get("K3_optionC", np.nan))
    if not np.isfinite(K3):
        raise RuntimeError("K3 unavailable. Ensure Option C cell defined phi_star+curvature or set K3_optionC.")
    C3, C2, C1 = 4/3, 3/4, 0.0
    eps = 0.10
    K2 = K3 * (C2 / C3)          # ~0.5625*K3
    K1 = K3 * ((eps) / C3)       # ~0.075*K3
    return K1, K2, K3

def get_Ki_by_channels():
    """
    Preferred: measure K_i from the same solver with channel-specific kernels.
    CONNECT HERE: define `solve_plateau_binding_channel(name, params)` and `H_phi_channel(phi, params)`.
    """
    if "solve_plateau_binding_channel" not in globals() or "H_phi_channel" not in globals():
        raise RuntimeError("Define solve_plateau_binding_channel(...) and H_phi_channel(...) in an earlier cell.")

    def run_channel(name, params):
        phi_star_i, Delta_eff_i = solve_plateau_binding_channel(name, params)  # <-- CONNECT HERE
        curv_i = local_curvature(lambda ph: H_phi_channel(ph, params), phi_star_i, h=1e-3)  # <-- CONNECT HERE
        Ki = stiffness_from(phi_star_i, curv_i)
        return ChannelResult(name, phi_star_i, Delta_eff_i, curv_i, Ki)

    # Example params; adjust to your kernels:
    ch_u1  = run_channel("U(1)", {"type":"abelian",    "g":0.40, "beta":1.2})
    ch_su2 = run_channel("SU(2)", {"type":"nonabelian","Nc":2,    "g":0.80, "beta":1.8})
    ch_su3 = run_channel("SU(3)", {"type":"nonabelian","Nc":3,    "g":1.00, "beta":2.2})

    return ch_u1.K, ch_su2.K, ch_su3.K

# ---------- Main flow ----------
if MODE == "casimir":
    K1, K2, K3 = get_Ki_by_casimir()
else:
    K1, K2, K3 = get_Ki_by_channels()

# Solve c_norm to match alpha_em at MZ (same-scale preview; RGE may adjust slightly)
c_norm = solve_cnorm_for_alpha_em(K1, K2, chi1, chi2, MAPPING, ALPHA_EM_INV_TARGET)
a1B, a2B, a3B = couplings_from_K(K1,K2,K3, chi1,chi2,chi3, MAPPING, c_norm)
aYB = (3.0/5.0)*a1B
sin2_preview = aYB / (aYB + a2B)

print("\n=== Gauge-channel stiffnesses (K_i) ===")
print(f"K1 (U1):  {K1:.6f}")
print(f"K2 (SU2): {K2:.6f}")
print(f"K3 (SU3): {K3:.6f}")

print("\n=== Single normalization solved from α_em(MZ) ===")
print(f"c_norm: {c_norm:.9f}   (mapping = {MAPPING})")

print("\n--- Same-scale PREVIEW (no RGE) ---")
print(summary_row("alpha_em_inv", 1.0/(1.0/(a2B) + 1.0/(aYB)), ALPHA_EM_INV_TARGET))
print(summary_row("sin^2(thetaW)", sin2_preview))
print(summary_row("alpha3 (≡ α_s?)", a3B))

# ---------- Optional: run to M_Z if RGE function exists ----------
def _has_RGE():
    return USE_RGE_IF_AVAILABLE and "run_to_MZ" in globals()

if _has_RGE():
    # Expect signature like: run_to_MZ(a1B,a2B,a3B, LambdaB=None) -> (a1_MZ,a2_MZ,a3_MZ)
    a1_MZ, a2_MZ, a3_MZ = run_to_MZ(a1B, a2B, a3B)
    aY_MZ = (3.0/5.0)*a1_MZ
    alpha_em_inv_MZ = 1.0 / ( 1.0/a2_MZ + 1.0/aY_MZ )
    sin2_MZ = aY_MZ / (aY_MZ + a2_MZ)

    print("\n--- After RGE to M_Z ---")
    print(summary_row("alpha_em_inv(MZ)", alpha_em_inv_MZ, ALPHA_EM_INV_TARGET))
    print(summary_row("sin^2(thetaW)(MZ)", sin2_MZ, 0.23122))
    print(summary_row("alpha_s(MZ)", a3_MZ, 0.11790))
else:
    print("\n(No RGE run: define run_to_MZ(...) earlier to produce PDG-scale comparisons.)")



=== Gauge-channel stiffnesses (K_i) ===
K1 (U1):  0.075000
K2 (SU2): 0.562500
K3 (SU3): 1.000000

=== Single normalization solved from α_em(MZ) ===
c_norm: 0.005372983   (mapping = invK)

--- Same-scale PREVIEW (no RGE) ---
alpha_em_inv         0.007815  vs  127.955000  Δ%= -99.994
sin^2(thetaW)        0.818182
alpha3 (≡ α_s?)      0.005373


TypeError: run_to_MZ() missing 2 required positional arguments: 'Lambda_B_GeV' and 'MZ_GeV'

> **Decision rule (closure test):** Without retuning the single normalization, do we get
> 1) \(\alpha(M_Z)\), \(\sin^2\theta_W(M_Z)\), \(\alpha_s(M_Z)\) within tolerance?  
> 2) \(\sqrt{\sigma}\approx 440\,\mathrm{MeV}\) and a within FLAG/MILC ranges?  
> If **no**, the stiffness dictionary needs revision (exponents, Bridge scale, or canonical mapping).

PERTURBATIVE_PORTON